In [1]:
import ee

# ee.Authenticate()
ee.Initialize()

In [2]:
import sys

sys.path.append(
    "C:/Users/ishaa/OneDrive/Desktop/DataLab/Syria/syria-economic-monitor/Phenolopy/scripts"
)
import phenolopy

In [3]:
sys.path.append(
    "C:/Users/ishaa/OneDrive/Desktop/DataLab/Syria/syria-economic-monitor/GEE_Zonal/src/gee_zonal"
)  # Replace with the actual path to the GEE_Zonal repository

In [4]:
from os.path import join, expanduser

# sys.path.append(join(expanduser("~"), "Repos", "GEE_Zonal", "src"))

# Data
import geopandas as gpd
import numpy as np

## Earth Engine APIs
import ee
from gee_helpers import gpd_to_gee

# from gee_zonal import gpd_to_gee
from geemap.conversion import *

import xarray as xr
from tqdm.notebook import tqdm
from dask.distributed import Client, LocalCluster
import requests

from plotnine import *


# sys.path.append(join(expanduser("~"), "Repos", "phenolopy", "scripts"))
# import phenolopy

sys.path.append(join(expanduser("~"), "Repos", "GOSTrocks", "src"))

In [33]:
start_period = ee.Date("2004-01-01")
end_period = ee.Date("2023-12-31")

terra = (
    ee.ImageCollection("MODIS/061/MOD13Q1")
    .select(["EVI", "SummaryQA", "DetailedQA"])
    .filterDate(start_period, end_period)
)
aqua = (
    ee.ImageCollection("MODIS/061/MYD13Q1")
    .select(["EVI", "SummaryQA", "DetailedQA"])
    .filterDate(start_period, end_period)
)

modis = terra.select("EVI").merge(aqua.select("EVI"))
modis = modis.sort("system:time_start")


def bitwiseExtract(value, fromBit, toBit=None):
    if toBit == None:
        toBit = fromBit
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)


# Applying the SummaryQA  and DetailedQA
def modisQA_mask(image):
    sqa = image.select("SummaryQA")
    dqa = image.select("DetailedQA")
    viQualityFlagsS = bitwiseExtract(sqa, 0, 1)
    viQualityFlagsD = bitwiseExtract(dqa, 0, 1)
    # viUsefulnessFlagsD = bitwiseExtract(dqa, 2, 5)
    viSnowIceFlagsD = bitwiseExtract(dqa, 14)
    viShadowFlagsD = bitwiseExtract(dqa, 15)
    # Good data, use with confidence
    mask = (
        viQualityFlagsS.eq(0)
        .And(viQualityFlagsD.eq(0))
        .And(viQualityFlagsS.eq(1))
        .And(viQualityFlagsD.eq(1))
        .And(viSnowIceFlagsD)
        .eq(0)
    )
    # .And(viShadowFlagsD).eq(0); # No shadow
    return image.updateMask(mask)


mod13q1_QC = terra.map(modisQA_mask)
myd13q1_QC = aqua.map(modisQA_mask)

mxd13q1_cleaned = mod13q1_QC.select("EVI").merge(myd13q1_QC.select("EVI"))
mxd13q1_cleaned_sorted = mxd13q1_cleaned.sort("system:time_start")

bool_dict = {
    "0": "ocean",
    "1": "non_crop",
    "2": "crop_irrigated",
    "3": "crop_rainfed",
}

lgrip30 = ee.ImageCollection(
    "projects/sat-io/open-datasets/GFSAD/LGRIP30"
).mosaic()  # .clip(geoj)
crop_data = lgrip30.select("b1").gt(1).rename("crop")

In [6]:
iso3 = "SYR"
country = "Syria"
release_type = "gbOpen"

adm = "ADM0"
geo_url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/"
res = requests.get(geo_url).json()
print("Reading " + res["gjDownloadURL"])
adm0_syr = gpd.read_file(res["gjDownloadURL"])

adm = "ADM1"
geo_url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/"
res = requests.get(geo_url).json()
print("Reading " + res["gjDownloadURL"])
adm1_syr = gpd.read_file(res["gjDownloadURL"])

adm = "ADM2"
geo_url = f"https://www.geoboundaries.org/api/current/{release_type}/{iso3}/{adm}/"
res = requests.get(geo_url).json()
print("Reading " + res["gjDownloadURL"])
adm2_syr = gpd.read_file(res["gjDownloadURL"])

Reading https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/SYR/ADM0/geoBoundaries-SYR-ADM0.geojson
Reading https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/SYR/ADM1/geoBoundaries-SYR-ADM1.geojson
Reading https://github.com/wmgeolab/geoBoundaries/raw/9469f09/releaseData/gbOpen/SYR/ADM2/geoBoundaries-SYR-ADM2.geojson


In [7]:
aoi = gpd_to_gee(adm0_syr)

In [34]:
scale_factor = 0.0001


# apply cropland mask to imageCollection
# def cropmask(img):
#   return img.clip(aoi) #.multiply(scale_factor) # unmask(-1)
def cropmask(img):
    return img.updateMask(crop_data).clip(aoi)  # .multiply(scale_factor) # unmask(-1)


mxd13q1 = mxd13q1_cleaned_sorted.map(cropmask)

In [9]:
ee.Initialize(opt_url="https://earthengine-highvolume.googleapis.com")

In [10]:
utm = utm = (
    "EPSG:32636"  # 'EPSG:4326' #"EPSG:32632"  #'EPSG:4326' 'SR-ORG:6974' 'EPSG:23847'
)

In [35]:
ds_all = []
scale_factor = 0.0001
years = range(2004, 2005)
months = range(5, 7)
for year in tqdm(years):
    for month in tqdm(months):
        modis_sel = mxd13q1.filter(ee.Filter.calendarRange(year, year, "year")).filter(
            ee.Filter.calendarRange(month, month, "month")
        )
        # modis_sel = mxd13q1.filterDate(f'{year}-{month}-01', f'{year}-{month+1}-01')
        ds = xr.open_dataset(
            modis_sel,
            engine="ee",
            geometry=aoi.geometry(),
            crs=utm,
            scale=250,
        )
        ds = ds * scale_factor
        ds = ds.compute()
        ds_all.append(ds)
    # print(f"Finished {year}")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
ds = xr.concat(ds_all, dim="time")

In [37]:
print(ds)

<xarray.Dataset> Size: 153MB
Dimensions:  (time: 7, X: 2303, Y: 2374)
Coordinates:
  * time     (time) datetime64[ns] 56B 2004-05-08 2004-05-16 ... 2004-06-25
  * X        (X) float64 18kB 7.559e+05 7.561e+05 ... 1.331e+06 1.331e+06
  * Y        (Y) float64 19kB 3.578e+06 3.579e+06 ... 4.171e+06 4.172e+06
Data variables:
    EVI      (time, X, Y) float32 153MB nan nan nan nan nan ... nan nan nan nan
Attributes:
    crs:      EPSG:32636


In [ ]:
# sum(1 for val in ds['EVI'] if np.isnan(val))

In [31]:
ds = ds.isel(Y=slice(None, None, -1))

In [38]:
for val in ds["EVI"]:
    print(val.values)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan

In [15]:
ds.to_netcdf(
    "C:\\Users\\ishaa\\OneDrive\\Desktop\\DataLab\\Syria\\syria-economic-monitor\\notebooks\\vegetation-conditions\\modis_evi_2020_2021_2months.nc"
)

In [16]:
netcdf_path = "C:\\Users\\ishaa\\OneDrive\\Desktop\\DataLab\\Syria\\syria-economic-monitor\\notebooks\\vegetation-conditions\\modis_evi_2020_2021_2months.nc"
ds = xr.open_dataset(netcdf_path)  # maradi
# ds = ds.isel(time=range(0,23))

In [17]:
ds

<xarray.Dataset> Size: 175MB
Dimensions:  (time: 8, X: 2303, Y: 2374)
Coordinates:
  * time     (time) datetime64[ns] 64B 2020-01-01 2020-01-09 ... 2020-02-26
  * X        (X) float64 18kB 7.559e+05 7.561e+05 ... 1.331e+06 1.331e+06
  * Y        (Y) float64 19kB 4.172e+06 4.171e+06 ... 3.579e+06 3.578e+06
Data variables:
    EVI      (time, X, Y) float32 175MB ...
Attributes:
    crs:      EPSG:32636

In [18]:
import multiprocessing

multiprocessing.cpu_count()

8

In [19]:
# Let's start simple with a LocalCluster that makes use of all the cores and RAM we have on a single machine
# cluster = LocalCluster()
# explicitly connect to the cluster we just created
# client = Client(cluster)
# client

# Dask Prep
workers = 50  # 50
cluster = LocalCluster(n_workers=workers)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 50
Total threads: 50,Total memory: 15.37 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:49831,Workers: 50
Dashboard: http://127.0.0.1:8787/status,Total threads: 50
Started: Just now,Total memory: 15.37 GiB
Comm: tcp://127.0.0.1:50120,Total threads: 1
Dashboard: http://127.0.0.1:50148/status,Memory: 314.75 MiB
Nanny: tcp://127.0.0.1:49834,


In [20]:
ds_all = []
years = range(2020, 2021)  # 2001 2024
for year in tqdm(years):
    ds_sel = ds.sel(time=ds.time.dt.year == year)
    dask_chunks = {"time": -1, "X": 500, "Y": 500}
    ds_sel = ds_sel.chunk(dask_chunks)
    ds_sel = ds_sel.rename_vars({"EVI": "veg_index"})
    ds_sel = phenolopy.remove_outliers(
        ds=ds_sel, method="median", user_factor=2, z_pval=0.05
    )
    ds_sel = ds_sel.chunk(dask_chunks)
    ds_sel = phenolopy.interpolate(ds=ds_sel, method="interpolate_na")
    ds_sel = phenolopy.smooth(
        ds=ds_sel, method="savitsky", window_length=3, polyorder=1
    )
    ds_sel = ds_sel.compute()
    ds_all.append(ds_sel)
    print(f"Finished {year}")

  0%|          | 0/1 [00:00<?, ?it/s]

Outlier removal method: median with a user factor of: 2
> Generated roll window size less than 3, setting to default (3).


c:\Users\ishaa\anaconda3\Anaconda\envs\GIS\lib\site-packages\xarray\groupers.py:326: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.


> Warning: dataset contains nan values. You may want to interpolate next.
> Outlier removal successful.

Interpolating dataset using method: interpolate_na.
> Warning: dataset still contains nan values. The first and/or last time slices may be empty.
> Interpolation successful.

Smoothing method: savitsky with window length: 3 and polyorder: 1.
> Warning: dataset contains nan values. You may want to interpolate next.
> Smoothing successful.



2024-10-15 10:38:29,841 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:50178 (pid=18240) exceeded 95% memory budget. Restarting...
2024-10-15 10:38:30,705 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:50099 (pid=24036) exceeded 95% memory budget. Restarting...
2024-10-15 10:38:31,319 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:50178' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('getitem-3121f3322ed454f877b46e751d75db7e', 0, 0.09999999999999998, 2.1), ('rechunk-merge-ec6f0f2ea3b46b923def44b8ba237010', 1, 0, 2), ('getitem-3121f3322ed454f877b46e751d75db7e', 1, 0.09999999999999998, 2.1), ('getitem-3121f3322ed454f877b46e751d75db7e', 0.09999999999999998, 0.09999999999999998, 2), ('getitem-3121f3322ed454f877b46e751d75db7e', 0.9, 0.09999999999999998, 1.9), ('getitem-3121f3322ed454f877b46e751d75db7e', 0, 0.09999999999999998, 2), ('rechunk-split-rechunk-merge-086ab3155d2164bca42827ecb3726b4c', 0

LinAlgError: SVD did not converge in Linear Least Squares

In [35]:
print(ds_all)

[<xarray.Dataset> Size: 88MB
Dimensions:  (time: 4, X: 2303, Y: 2374)
Coordinates:
  * time     (time) datetime64[ns] 32B 2020-01-01 2020-01-09 ... 2020-01-25
  * X        (X) float64 18kB 7.559e+05 7.561e+05 ... 1.331e+06 1.331e+06
  * Y        (Y) float64 19kB 3.578e+06 3.579e+06 ... 4.171e+06 4.172e+06
Data variables:
    EVI      (time, X, Y) float32 87MB nan nan nan nan nan ... nan nan nan nan
Attributes:
    crs:      EPSG:32636, <xarray.Dataset> Size: 88MB
Dimensions:  (time: 4, X: 2303, Y: 2374)
Coordinates:
  * time     (time) datetime64[ns] 32B 2020-02-02 2020-02-10 ... 2020-02-26
  * X        (X) float64 18kB 7.559e+05 7.561e+05 ... 1.331e+06 1.331e+06
  * Y        (Y) float64 19kB 3.578e+06 3.579e+06 ... 4.171e+06 4.172e+06
Data variables:
    EVI      (time, X, Y) float32 87MB nan nan nan nan nan ... nan nan nan nan
Attributes:
    crs:      EPSG:32636]


In [ ]:
"""ds_all_WI = []
ds_all_WithoutI = []
years = range(2020, 2021)  # 2001 2024
for year in tqdm(years):
    ds_sel = ds.sel(time=ds.time.dt.year == year)
    dask_chunks = {"time": -1, "X": 100, "Y": 100}
    ds_sel = ds_sel.chunk(dask_chunks)
    ds_sel = ds_sel.rename_vars({"EVI": "veg_index"})
    ds_sel = phenolopy.remove_outliers(
        ds=ds_sel, method="median", user_factor=2, z_pval=0.05
    )
    ds_sel = ds_sel.where(~np.isnan(ds_sel['veg_index']))
    ds_sel = ds_sel.chunk(dask_chunks)
    ds_sel1 = phenolopy.interpolate(ds=ds_sel, method="interpolate_na")
    ds_sel
    ds_sel = phenolopy.smooth(
        ds=ds_sel, method="savitsky", window_length=3, polyorder=1
    )
    ds_sel = ds_sel.compute()
    ds_sel1 = ds_sel1.compute()
    ds_all_WI.append(ds_sel1)
    ds_all_WithoutI.append(ds_sel)
    print(f"Finished {year}")"""

In [44]:
s = ds_all_WithoutI[0]["veg_index"].to_numpy().flatten()

In [47]:
sum(s)

np.float32(nan)

In [48]:
s1 = s[:10000]

In [49]:
sum(1 for val in s1 if not np.isnan(val))

264

In [51]:
s = ds_all_WI[0]["veg_index"].to_numpy().flatten()
s1 = s[60000:80000]
sum(1 for val in s1 if not np.isnan(val))

1078

In [54]:
s2 = phenolopy.smooth(
    ds=ds_all_WithoutI[0], method="savitsky", window_length=3, polyorder=1
)

Smoothing method: savitsky with window length: 3 and polyorder: 1.


LinAlgError: SVD did not converge in Linear Least Squares

In [54]:
sum(1 for val in ds_all_WI[0]["veg_index"] if np.isnan(val))

46

In [21]:
len(ds_all)

4

In [ ]:
import math

math.isnan(ds_all[0]).sum()

In [75]:
ds_all[0].isnull().sum()

<xarray.Dataset> Size: 8B
Dimensions:  ()
Data variables:
    EVI      int64 8B 17703213

In [76]:
ds_all[0]

<xarray.Dataset> Size: 88MB
Dimensions:  (time: 4, X: 2303, Y: 2374)
Coordinates:
  * time     (time) datetime64[ns] 32B 2020-01-01 2020-01-09 ... 2020-01-25
  * X        (X) float64 18kB 7.559e+05 7.561e+05 ... 1.331e+06 1.331e+06
  * Y        (Y) float64 19kB 3.578e+06 3.579e+06 ... 4.171e+06 4.172e+06
Data variables:
    EVI      (time, X, Y) float32 87MB nan nan nan nan nan ... nan nan nan nan
Attributes:
    crs:      EPSG:32636

In [ ]:
ds2 = xr.concat(ds_all, dim="time")

In [56]:
# convert ds to zarr file
ds_temp = xr.open_dataset(netcdf_path)
ds_temp.to_zarr("2020_2021_2months_modis.zarr", mode="w")

In [ ]:
# Open the Zarr file
ds_zarr = xr.open_zarr("2020_2021_2months_modis.zarr")

# Now you can explore the dataset
print(ds_zarr)

In [58]:
ds_geo = xr.open_zarr("2020_2021_2months_modis.zarr")
ds_geo

<xarray.Dataset> Size: 175MB
Dimensions:  (time: 8, X: 2303, Y: 2374)
Coordinates:
  * X        (X) float64 18kB 7.559e+05 7.561e+05 ... 1.331e+06 1.331e+06
  * Y        (Y) float64 19kB 4.172e+06 4.171e+06 ... 3.579e+06 3.578e+06
  * time     (time) datetime64[ns] 64B 2020-01-01 2020-01-09 ... 2020-02-26
Data variables:
    EVI      (time, X, Y) float32 175MB dask.array<chunksize=(1, 576, 594), meta=np.ndarray>
Attributes:
    crs:      EPSG:32636

In [59]:
ds_monthly = ds_geo.resample(time="1ME").mean()

In [61]:
start_reference = "2020-01-01"
end_reference = "2020-02-28"

In [62]:
ds_ref = ds_monthly.sel(time=slice(None, end_reference))  # ds_geo
ds_filt = ds_monthly.sel(time=ds_monthly.time.dt.year == 2020)

ds_2020_01 = ds_filt.sel(time=ds_filt.time.dt.month == 1).isel(time=0)
ds_ref_01_mean = ds_ref.sel(time=ds_ref.time.dt.month == 1).mean(dim="time")
ds_ref_01_sdt = ds_ref.sel(time=ds_ref.time.dt.month == 1).std(dim="time")

ds_diff = ds_2020_01 - ds_ref_01_mean
ds_zscore = ds_diff / ds_ref_01_sdt

In [63]:
ds_zscore = ds_zscore.compute()

In [67]:
print(ds_zscore)

<xarray.Dataset> Size: 22MB
Dimensions:  (X: 2303, Y: 2374)
Coordinates:
  * X        (X) float64 18kB 7.559e+05 7.561e+05 ... 1.331e+06 1.331e+06
  * Y        (Y) float64 19kB 4.172e+06 4.171e+06 ... 3.579e+06 3.578e+06
    time     datetime64[ns] 8B 2020-01-31
Data variables:
    EVI      (X, Y) float32 22MB nan nan nan nan nan nan ... nan nan nan nan nan


In [66]:
ds_zscore["EVI"].rio.write_nodata(np.nan, inplace=True)
ds_zscore["EVI"].rio.to_raster(
    "C:\\Users\\ishaa\\OneDrive\\Desktop\\DataLab\\Syria\\syria-economic-monitor\\notebooks\\vegetation-conditions\\zcore_dea_2023_09.tif"
)

MissingSpatialDimensionError: y dimension not found. 'rio.set_spatial_dims()' or using 'rename()' to change the dimension name to 'y' can address this. Data variable: EVI